# COMP4211 Programming Assignment 2

**Name: FONG, Ho Yin**  
**SID: 20599045**

### 4.2.1  Dataset and Dataloader

Your first task is to implement a custom Dataset class for the Face Dataset. You are required to preprocess the images by: (1) resizing all the images to a size of 32x32, and (2) converting them to single-channel images. The Dataset.getitem() function should return two 32x32, single-channel images and one target label.

Create the Dataset and Dataloader objects for train.csv, valid.csv and test.csv. You are suggested to use a batch size of 128 for model training.

In [1]:
PATH = './'

import pandas as pd
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import os
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

class ImageDataset(Dataset):
    def __init__(self, csv_file, index_dir, transform=None):
        self.info_df = pd.read_csv(csv_file)    
        self.index_dir = index_dir
        self.transform = transform

    def __getitem__(self, idx):
        index_file = open(self.index_dir, 'r')
        file_lines = index_file.readlines()
        id1 = self.info_df.iloc[idx, 0]
        image_path1 = file_lines[id1-1]
        image_path1 = os.path.join(PATH, image_path1[image_path1.find(',')+1:image_path1.find('\n')])
        id2 = self.info_df.iloc[idx,1]
        image_path2 = file_lines[id2-1]
        image_path2 = os.path.join(PATH, image_path2[image_path2.find(',')+1:image_path1.find('\n')])
        image1 = Image.open(image_path1).convert('L')
        image2 = Image.open(image_path2).convert('L')
        label = np.array(self.info_df.iloc[idx, 2].astype(np.float32))
        
        if self.transform is not None:
            image1, image2 = self.transform(image1), self.transform(image2)
        
        return image1, image2, label

    def __len__(self):
        return len(self.info_df)

transform = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.ToTensor(), 
     transforms.Normalize((0.5,), (0.5, )) ])
    
train_dataset = ImageDataset(PATH + 'train.csv', PATH + 'index.txt', transform)
valid_dataset = ImageDataset(PATH + 'valid.csv', PATH + 'index.txt', transform)
test_dataset = ImageDataset(PATH + 'test.csv', PATH + 'index.txt', transform)

In [2]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

### 4.2.2 Siamese Convolutional Neural Network

#### Convolutional Layers

The convolutional layers take an image as input and produce a hidden state vector as output. Here, you need to build a stack of convolutional layers according to the network architecture illustrated in Table 2. In addition, each convolutional layer is followed by batch normalization and a ReLU activation. Notice that the kernel size of the final average pooling layer is not provided. Given that the size of the output vector is [1 x 512], calculate the correct kernel size for the final average pooling layer and implement it in your model.

**[Q1]** What is the correct kernel size in the average pooling layer? Explain your answer. (Hint: You may assume that there is a flattening operation after the average pooling layer.)

**[Q2]** What is dropout in deep learning? Why does it work? Explain it briefly

**[Q3]** What is the total number of trainable parameters in this Siamese network model?

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)     # Conv 32 3x3 kernels, 1 stride 1 padding
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)    # Conv 32 3x3 kernels, 1 stride 1 padding
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                                   # Pool 32 2x2 kernels, 2 stirde 0 padding
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)    # Conv 64 3x3 kernels, 1 stride 1 padding
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)   # Conv 128 3x3 kernels, 1 stride 1 padding
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)  # Conv 256 3x3 kernels, 1 stride 1 padding
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)  # Conv 512 3x3 kernels, 1 stride 1 padding
        self.pool2 = nn.AvgPool2d(kernel_size=16)                                            # Pool 1 16x16 kernels, 1 stirde 0 padding
        
        self.norm1 = nn.BatchNorm2d(32) 
        self.norm2 = nn.BatchNorm2d(32) 
        self.norm3 = nn.BatchNorm2d(64)
        self.norm4 = nn.BatchNorm2d(128)
        self.norm5 = nn.BatchNorm2d(256)
        self.norm6 = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(512, 512)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512,1)

    def forward(self, x1, x2):
        x1 = self.norm1(F.relu(self.conv1(x1)))
        x1 = self.norm2(F.relu(self.conv2(x1)))
        x1 = self.pool1(x1)
        x1 = self.norm3(F.relu(self.conv3(x1)))
        x1 = self.norm4(F.relu(self.conv4(x1)))
        x1 = self.norm5(F.relu(self.conv5(x1)))
        x1 = self.norm6(F.relu(self.conv6(x1)))
        x1 = self.pool2(x1)
        x2 = self.norm1(F.relu(self.conv1(x2)))
        x2 = self.norm2(F.relu(self.conv2(x2)))
        x2 = self.pool1(x2)
        x2 = self.norm3(F.relu(self.conv3(x2)))
        x2 = self.norm4(F.relu(self.conv4(x2)))
        x2 = self.norm5(F.relu(self.conv5(x2)))
        x2 = self.norm6(F.relu(self.conv6(x2)))
        x2 = self.pool2(x2)
        x1 = x1.view(-1, 512)
        x2 = x2.view(-1, 512)
        x = abs(x2 - x1)
        x = self.drop(F.relu(self.fc1(x)))
        x = torch.sigmoid(self.fc2(x))
        return x


In [4]:
from torchsummaryX import summary

model = Net().cuda()
summary(model, torch.zeros((128, 1, 32, 32)).cuda(), torch.zeros((128, 1, 32, 32)).cuda())

              Kernel Shape        Output Shape    Params    Mult-Adds
Layer                                                                
0_conv1      [1, 32, 3, 3]   [128, 32, 32, 32]     320.0     294.912k
1_norm1               [32]   [128, 32, 32, 32]      64.0         32.0
2_conv2     [32, 32, 3, 3]   [128, 32, 32, 32]    9.248k    9.437184M
3_norm2               [32]   [128, 32, 32, 32]      64.0         32.0
4_pool1                  -   [128, 32, 16, 16]         -            -
5_conv3     [32, 64, 3, 3]   [128, 64, 16, 16]   18.496k    4.718592M
6_norm3               [64]   [128, 64, 16, 16]     128.0         64.0
7_conv4    [64, 128, 3, 3]  [128, 128, 16, 16]   73.856k   18.874368M
8_norm4              [128]  [128, 128, 16, 16]     256.0        128.0
9_conv5   [128, 256, 3, 3]  [128, 256, 16, 16]  295.168k   75.497472M
10_norm5             [256]  [128, 256, 16, 16]     512.0        256.0
11_conv6  [256, 512, 3, 3]  [128, 512, 16, 16]  1.18016M  301.989888M
12_norm6            

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1,"[1, 32, 3, 3]","[128, 32, 32, 32]",320.0,294912.0
1_norm1,[32],"[128, 32, 32, 32]",64.0,32.0
2_conv2,"[32, 32, 3, 3]","[128, 32, 32, 32]",9248.0,9437184.0
3_norm2,[32],"[128, 32, 32, 32]",64.0,32.0
4_pool1,-,"[128, 32, 16, 16]",NaN,NaN
5_conv3,"[32, 64, 3, 3]","[128, 64, 16, 16]",18496.0,4718592.0
6_norm3,[64],"[128, 64, 16, 16]",128.0,64.0
7_conv4,"[64, 128, 3, 3]","[128, 128, 16, 16]",73856.0,18874368.0
8_norm4,[128],"[128, 128, 16, 16]",256.0,128.0


**[Q4]** If we replace the absolute difference aggregation function by concatenation (i.e., concatenating the hidden state vectors of the two input images into one before feeding into the fully
connected layers), what is the change in the total number of trainable parameters when compared
to the original setting?

In [5]:
class ConcatNet(nn.Module):
    def __init__(self):
        super(ConcatNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)     # Conv 32 3x3 kernels, 1 stride 1 padding
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)    # Conv 32 3x3 kernels, 1 stride 1 padding
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                                   # Pool 32 2x2 kernels, 2 stirde 0 padding
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)    # Conv 64 3x3 kernels, 1 stride 1 padding
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)   # Conv 128 3x3 kernels, 1 stride 1 padding
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)  # Conv 256 3x3 kernels, 1 stride 1 padding
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)  # Conv 512 3x3 kernels, 1 stride 1 padding
        self.pool2 = nn.AvgPool2d(kernel_size=16)                                            # Pool 1 16x16 kernels, 1 stirde 0 padding
        
        self.norm1 = nn.BatchNorm2d(32) 
        self.norm2 = nn.BatchNorm2d(32) 
        self.norm3 = nn.BatchNorm2d(64)
        self.norm4 = nn.BatchNorm2d(128)
        self.norm5 = nn.BatchNorm2d(256)
        self.norm6 = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(1024, 512)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512,1)
        
    def forward(self, x1, x2):
        x1 = self.norm1(F.relu(self.conv1(x1)))
        x1 = self.norm2(F.relu(self.conv2(x1)))
        x1 = self.pool1(x1)
        x1 = self.norm3(F.relu(self.conv3(x1)))
        x1 = self.norm4(F.relu(self.conv4(x1)))
        x1 = self.norm5(F.relu(self.conv5(x1)))
        x1 = self.norm6(F.relu(self.conv6(x1)))
        x1 = self.pool2(x1)
        x2 = self.norm1(F.relu(self.conv1(x2)))
        x2 = self.norm2(F.relu(self.conv2(x2)))
        x2 = self.pool1(x2)
        x2 = self.norm3(F.relu(self.conv3(x2)))
        x2 = self.norm4(F.relu(self.conv4(x2)))
        x2 = self.norm5(F.relu(self.conv5(x2)))
        x2 = self.norm6(F.relu(self.conv6(x2)))
        x2 = self.pool2(x2)
        x1 = x1.view(-1, 512)
        x2 = x2.view(-1, 512)
        x = torch.cat((x1,x2),1)
        x = self.drop(F.relu(self.fc1(x)))
        x = torch.sigmoid(self.fc2(x))
        return x

Q4model = ConcatNet().cuda()
print(summary(Q4model, torch.zeros((1, 1, 32, 32)).cuda(), torch.zeros((1, 1, 32, 32)).cuda()))

              Kernel Shape      Output Shape    Params    Mult-Adds
Layer                                                              
0_conv1      [1, 32, 3, 3]   [1, 32, 32, 32]     320.0     294.912k
1_norm1               [32]   [1, 32, 32, 32]      64.0         32.0
2_conv2     [32, 32, 3, 3]   [1, 32, 32, 32]    9.248k    9.437184M
3_norm2               [32]   [1, 32, 32, 32]      64.0         32.0
4_pool1                  -   [1, 32, 16, 16]         -            -
5_conv3     [32, 64, 3, 3]   [1, 64, 16, 16]   18.496k    4.718592M
6_norm3               [64]   [1, 64, 16, 16]     128.0         64.0
7_conv4    [64, 128, 3, 3]  [1, 128, 16, 16]   73.856k   18.874368M
8_norm4              [128]  [1, 128, 16, 16]     256.0        128.0
9_conv5   [128, 256, 3, 3]  [1, 256, 16, 16]  295.168k   75.497472M
10_norm5             [256]  [1, 256, 16, 16]     512.0        256.0
11_conv6  [256, 512, 3, 3]  [1, 512, 16, 16]  1.18016M  301.989888M
12_norm6             [512]  [1, 512, 16, 16]    

### 4.2.3 Training and Validation
The model is trained by minimizing the binary cross-entropy loss1 using the Adam optimizer with the default setting. You are expected to train your model for 20 epochs. During training, you are required to record the training and validation losses after every 10 steps (one step means
one gradient update). Plot the training and validation loss curves versus the number of steps in the same graph using Tensorboard or matplotlib.

**[Q5]** Paste the screenshot of the training and validation loss curves and report the final training and validation losses obtained. According to your plot, when should you stop training? Why?

In [6]:
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to {save_path}')

def load_checkpoint(save_path, model, optimizer):
    save_path = save_path 
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from {save_path}, with val loss: {val_loss}')
    return val_loss


def TRAIN(net, train_loader, valid_loader,  num_epochs, criterion, optimizer, val_loss, device, save_name, writer = None):
    
    if val_loss==None:
        best_val_loss = float("Inf")  
    else: 
        best_val_loss=val_loss
        print('Resume training')
        
    step = 0
    running_loss = 0
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        net.train()
        #training loop
        for input1, input2, labels in tqdm(train_loader):
            
            input1 = input1.to(device)
            input2 = input2.to(device)
            labels = labels.to(device)

            outputs = net(input1, input2)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            step += 1
            
            running_loss += loss.item()
            
            if (step%10 == 0):
                # validation every 10 steps
                with torch.no_grad():
                    net.eval()
                    valid_running_loss = 0
                    for val_input1, val_input2, val_labels in valid_loader:

                        val_input1 = val_input1.to(device)
                        val_input2 = val_input2.to(device)
                        val_labels = val_labels.to(device)

                        val_outputs = net(val_input1, val_input2)
                        val_labels = val_labels.unsqueeze(1)
                        val_loss = criterion(val_outputs, val_labels)

                        valid_running_loss += val_loss.item()

                    valid_loss = valid_running_loss / len(valid_loader)
                    
                train_loss = running_loss / 10
                print('Step {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'
                          .format(step, train_loss, valid_loss))
                if (writer != None):
                    writer.add_scalars('Loss', {'train': train_loss, 'valid': valid_loss}, step)
                running_loss = 0
                
                if valid_loss < best_val_loss:
                    best_val_loss = valid_loss
                    save_checkpoint(save_name, net, optimizer, best_val_loss)
        
        print('Epoch [{}/{}] finished'.format(epoch+1, num_epochs))

    print('Finished Training')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 20
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
model = model.to(device)          
best_val_loss = None
save_path = f'net.pt'
writer = SummaryWriter(os.path.join(PATH, 'runs/hr'))

TRAIN(model, train_loader, valid_loader, num_epochs, criterion, optimizer, best_val_loss, device, save_path, writer)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


  0%|          | 0/39 [00:00<?, ?it/s]

Step 10, Training Loss: 0.7034, Validation Loss: 0.6911
Model saved to net.pt
Step 20, Training Loss: 0.6964, Validation Loss: 0.6921
Step 30, Training Loss: 0.6994, Validation Loss: 0.6879
Model saved to net.pt
Epoch [1/20] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 40, Training Loss: 0.6882, Validation Loss: 0.7044
Step 50, Training Loss: 0.6866, Validation Loss: 0.6890
Step 60, Training Loss: 0.6866, Validation Loss: 0.6767
Model saved to net.pt
Step 70, Training Loss: 0.6952, Validation Loss: 0.6980
Epoch [2/20] finished


...

Step 670, Training Loss: 0.3539, Validation Loss: 0.4862
Step 680, Training Loss: 0.3164, Validation Loss: 0.4908
Step 690, Training Loss: 0.2995, Validation Loss: 0.4948
Step 700, Training Loss: 0.3090, Validation Loss: 0.4801
Epoch [18/20] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 710, Training Loss: 0.2620, Validation Loss: 0.4996
Step 720, Training Loss: 0.2957, Validation Loss: 0.5402
Step 730, Training Loss: 0.2863, Validation Loss: 0.5060
Step 740, Training Loss: 0.3026, Validation Loss: 0.5142
Epoch [19/20] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 750, Training Loss: 0.2465, Validation Loss: 0.5422
Step 760, Training Loss: 0.2478, Validation Loss: 0.5235
Step 770, Training Loss: 0.2664, Validation Loss: 0.5328
Step 780, Training Loss: 0.2508, Validation Loss: 0.4955
Epoch [20/20] finished
Finished Training


In [8]:
%load_ext tensorboard
%tensorboard --logdir './runs'

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 13132.

### 4.2.4 Evaluation
Output the prediction of the validation set as 1 if the similarity score exceeds a predefined threshold θ, and as 0 otherwise. Write a helper function to search for the optimal θ that maximizes the validation accuracy.

**[Q6]** Report the optimal threshold θ and the corresponding validation accuracy.

**[Q7]** Suppose you are given a photo application used to identify similar-looking family members and a face authentication application used for mobile banking. Should you set a relatively high or low threshold for each of the applications? Why?


In [9]:
load_checkpoint(save_path, model, optimizer)

def find_threshold(net, valid_loader, device, threshold):
    with torch.no_grad():
        net.eval()
        running_corrects = 0
        for input1, input2, labels in valid_loader:

            input1 = input1.to(device)
            input2 = input2.to(device)
            labels = labels.to(device)

            outputs = net(input1,input2)
            preds = (outputs > threshold)
            running_corrects += torch.sum(preds == labels.data)
            
        valid_acc = running_corrects / float(len(valid_loader.dataset))
    return valid_acc

for threshold in range(0, 100, 5):
    accuracy = find_threshold(model, valid_loader, device, threshold/100)
    print('For threshold = {:.2f}, Accuracy = {:.4f}'.format(threshold/100, accuracy))

Model loaded from net.pt, with val loss: 0.4668879707654317
For threshold = 0.00, Accuracy = 64.0000
For threshold = 0.05, Accuracy = 80.8743
For threshold = 0.10, Accuracy = 84.2514
For threshold = 0.15, Accuracy = 86.5200
For threshold = 0.20, Accuracy = 87.8971
For threshold = 0.25, Accuracy = 88.5543
For threshold = 0.30, Accuracy = 89.0171
For threshold = 0.35, Accuracy = 88.9429
For threshold = 0.40, Accuracy = 89.0286
For threshold = 0.45, Accuracy = 88.5886
For threshold = 0.50, Accuracy = 88.6743
For threshold = 0.55, Accuracy = 88.9657
For threshold = 0.60, Accuracy = 86.6343
For threshold = 0.65, Accuracy = 86.0229
For threshold = 0.70, Accuracy = 84.1314
For threshold = 0.75, Accuracy = 80.7771
For threshold = 0.80, Accuracy = 77.1029
For threshold = 0.85, Accuracy = 67.8686
For threshold = 0.90, Accuracy = 59.8914
For threshold = 0.95, Accuracy = 58.1714


### 4.2.5 Improving your Model
Now, improve your model for better validation performance. You are required to make changes in at least two of the five categories (A-E) defined below. You may duplicate your previous code and modify it from there. Alternatively, you may also modify your previous code to accept new arguments for different configurations. Make sure that your changes are clearly shown in the code (e.g., you can put a comment like ‘# Q8A’ in your code to indicate the changes for category A) and do not overwrite your work for the previous tasks (Section 4.2.1 – 4.2.4). In addition to the changes listed below, you may make further changes to the model or training procedure to maximize the validation performance.

**[Q8]** Describe your changes and compare the validation performance with the original setting. Discuss why the changes improve model performance. You are expected to use visual aids like a graph or a table to explain your answer.



In [10]:
# Q8A: Optimization

modified_model1 = Net().cuda()
modified_model1 = modified_model1.to(device)

# First set of parameters: 15 epoch, 0.0005 learning rate
modified_num_epochs = 15
modified_optimizer = optim.Adam(modified_model1.parameters(), lr=0.0005, weight_decay=0)

best_val_loss = None
criterion = nn.BCELoss()
save_path1 = f'modify_net1.pt'
modify_writer1 = SummaryWriter(os.path.join(PATH, 'runs/hr1'))

TRAIN(modified_model1, train_loader, valid_loader, modified_num_epochs, criterion, modified_optimizer, best_val_loss, device, save_path1, modify_writer1)

  0%|          | 0/39 [00:00<?, ?it/s]

Step 10, Training Loss: 0.6986, Validation Loss: 0.6920
Model saved to modify_net1.pt
Step 20, Training Loss: 0.6914, Validation Loss: 0.6925
Step 30, Training Loss: 0.6880, Validation Loss: 0.6829
Model saved to modify_net1.pt
Epoch [1/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 40, Training Loss: 0.6816, Validation Loss: 0.6722
Model saved to modify_net1.pt
Step 50, Training Loss: 0.6860, Validation Loss: 0.6726
Step 60, Training Loss: 0.6804, Validation Loss: 0.6675
Model saved to modify_net1.pt
Step 70, Training Loss: 0.6752, Validation Loss: 0.6653
Model saved to modify_net1.pt
Epoch [2/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

...

Step 470, Training Loss: 0.3250, Validation Loss: 0.5384
Step 480, Training Loss: 0.2756, Validation Loss: 0.5235
Step 490, Training Loss: 0.2552, Validation Loss: 0.5002
Step 500, Training Loss: 0.2520, Validation Loss: 0.5275
Epoch [13/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 510, Training Loss: 0.2923, Validation Loss: 0.5028
Step 520, Training Loss: 0.2443, Validation Loss: 0.5356
Step 530, Training Loss: 0.2297, Validation Loss: 0.5341
Step 540, Training Loss: 0.2348, Validation Loss: 0.5258
Epoch [14/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 550, Training Loss: 0.2069, Validation Loss: 0.5270
Step 560, Training Loss: 0.1804, Validation Loss: 0.4978
Step 570, Training Loss: 0.1786, Validation Loss: 0.6109
Step 580, Training Loss: 0.1770, Validation Loss: 0.5933
Epoch [15/15] finished
Finished Training


In [11]:
# Accuracy calculation for first set of parameters
load_checkpoint(save_path1, modified_model1, optimizer)
for threshold in range(0, 100, 5):
    accuracy = find_threshold(modified_model1, valid_loader, device, threshold/100)
    print('For threshold = {:.2f}, Accuracy = {:.4f}'.format(threshold/100, accuracy))

Model loaded from modify_net1.pt, with val loss: 0.46266648670037586
For threshold = 0.00, Accuracy = 64.0000
For threshold = 0.05, Accuracy = 86.4343
For threshold = 0.10, Accuracy = 88.6057
For threshold = 0.15, Accuracy = 90.1771
For threshold = 0.20, Accuracy = 91.2743
For threshold = 0.25, Accuracy = 91.1886
For threshold = 0.30, Accuracy = 91.7371
For threshold = 0.35, Accuracy = 91.4800
For threshold = 0.40, Accuracy = 92.4800
For threshold = 0.45, Accuracy = 91.4800
For threshold = 0.50, Accuracy = 91.5886
For threshold = 0.55, Accuracy = 89.7200
For threshold = 0.60, Accuracy = 87.7429
For threshold = 0.65, Accuracy = 85.2171
For threshold = 0.70, Accuracy = 82.8857
For threshold = 0.75, Accuracy = 77.9086
For threshold = 0.80, Accuracy = 73.6743
For threshold = 0.85, Accuracy = 69.4286
For threshold = 0.90, Accuracy = 63.3314
For threshold = 0.95, Accuracy = 59.5486


In [12]:
# Second set of parameters: 15 epoch, 0.002 learning rate

modified_model2 = Net().cuda()
modified_model2 = modified_model2.to(device)

modified_num_epochs = 15
modified_optimizer = optim.Adam(modified_model2.parameters(), lr=0.002)
modify_writer2 = SummaryWriter(os.path.join(PATH, 'runs/hr2'))
save_path2 = f'modify_net2.pt'

TRAIN(modified_model2, train_loader, valid_loader, modified_num_epochs, criterion, modified_optimizer, best_val_loss, device, save_path2, modify_writer2)

  0%|          | 0/39 [00:00<?, ?it/s]

Step 10, Training Loss: 0.7537, Validation Loss: 0.6926
Model saved to modify_net2.pt
Step 20, Training Loss: 0.6990, Validation Loss: 0.6896
Model saved to modify_net2.pt
Step 30, Training Loss: 0.7008, Validation Loss: 0.6769
Model saved to modify_net2.pt
Epoch [1/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 40, Training Loss: 0.6859, Validation Loss: 0.6738
Model saved to modify_net2.pt
Step 50, Training Loss: 0.6836, Validation Loss: 0.6879
Step 60, Training Loss: 0.6859, Validation Loss: 0.6591
Model saved to modify_net2.pt
Step 70, Training Loss: 0.6817, Validation Loss: 0.6719
Epoch [2/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

...

Step 470, Training Loss: 0.4454, Validation Loss: 0.5673
Step 480, Training Loss: 0.4076, Validation Loss: 0.5110
Step 490, Training Loss: 0.4173, Validation Loss: 0.5048
Step 500, Training Loss: 0.4056, Validation Loss: 0.4795
Model saved to modify_net2.pt
Epoch [13/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 510, Training Loss: 0.3763, Validation Loss: 0.4812
Step 520, Training Loss: 0.3398, Validation Loss: 0.4999
Step 530, Training Loss: 0.3415, Validation Loss: 0.4813
Step 540, Training Loss: 0.3334, Validation Loss: 0.4814
Epoch [14/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 550, Training Loss: 0.3231, Validation Loss: 0.4717
Model saved to modify_net2.pt
Step 560, Training Loss: 0.2822, Validation Loss: 0.5439
Step 570, Training Loss: 0.3352, Validation Loss: 0.5332
Step 580, Training Loss: 0.3188, Validation Loss: 0.5513
Epoch [15/15] finished
Finished Training


In [13]:
# Accuracy calculation for second set of parameters
load_checkpoint(save_path2, modified_model2, optimizer)
for threshold in range(0, 100, 5):
    accuracy = find_threshold(modified_model2, valid_loader, device, threshold/100)
    print('For threshold = {:.2f}, Accuracy = {:.4f}'.format(threshold/100, accuracy))

Model loaded from modify_net2.pt, with val loss: 0.47173341115315753
For threshold = 0.00, Accuracy = 64.0000
For threshold = 0.05, Accuracy = 83.1771
For threshold = 0.10, Accuracy = 87.1771
For threshold = 0.15, Accuracy = 88.5543
For threshold = 0.20, Accuracy = 89.7257
For threshold = 0.25, Accuracy = 90.4571
For threshold = 0.30, Accuracy = 90.3829
For threshold = 0.35, Accuracy = 91.7486
For threshold = 0.40, Accuracy = 91.2971
For threshold = 0.45, Accuracy = 91.0286
For threshold = 0.50, Accuracy = 90.2114
For threshold = 0.55, Accuracy = 89.3829
For threshold = 0.60, Accuracy = 88.7086
For threshold = 0.65, Accuracy = 87.5486
For threshold = 0.70, Accuracy = 86.9143
For threshold = 0.75, Accuracy = 84.1314
For threshold = 0.80, Accuracy = 81.7886
For threshold = 0.85, Accuracy = 77.3714
For threshold = 0.90, Accuracy = 72.1371
For threshold = 0.95, Accuracy = 61.8914


In [14]:
# Third set of parameters: 15 epoch, 0.00075 learning rate

modified_model3 = Net().cuda()
modified_model3 = modified_model3.to(device)

modified_num_epochs = 15
modified_optimizer = optim.Adam(modified_model3.parameters(), lr=0.00075)
modify_writer3 = SummaryWriter(os.path.join(PATH, 'runs/hr3'))
save_path3 = f'modify_net3.pt'

TRAIN(modified_model3, train_loader, valid_loader, modified_num_epochs, criterion, modified_optimizer, best_val_loss, device, save_path3, modify_writer3)

  0%|          | 0/39 [00:00<?, ?it/s]

Step 10, Training Loss: 0.6970, Validation Loss: 0.6884
Model saved to modify_net3.pt
Step 20, Training Loss: 0.6954, Validation Loss: 0.6823
Model saved to modify_net3.pt
Step 30, Training Loss: 0.6887, Validation Loss: 0.6845
Epoch [1/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 40, Training Loss: 0.6863, Validation Loss: 0.6753
Model saved to modify_net3.pt
Step 50, Training Loss: 0.6861, Validation Loss: 0.6756
Step 60, Training Loss: 0.6817, Validation Loss: 0.6667
Model saved to modify_net3.pt
Step 70, Training Loss: 0.6654, Validation Loss: 0.6562
Model saved to modify_net3.pt
Epoch [2/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

...

Step 470, Training Loss: 0.4160, Validation Loss: 0.4950
Step 480, Training Loss: 0.3791, Validation Loss: 0.4760
Step 490, Training Loss: 0.3411, Validation Loss: 0.5015
Step 500, Training Loss: 0.3295, Validation Loss: 0.5386
Epoch [13/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 510, Training Loss: 0.3543, Validation Loss: 0.4941
Step 520, Training Loss: 0.3078, Validation Loss: 0.4894
Step 530, Training Loss: 0.3117, Validation Loss: 0.5058
Step 540, Training Loss: 0.3166, Validation Loss: 0.5059
Epoch [14/15] finished


  0%|          | 0/39 [00:00<?, ?it/s]

Step 550, Training Loss: 0.3108, Validation Loss: 0.5191
Step 560, Training Loss: 0.2808, Validation Loss: 0.5213
Step 570, Training Loss: 0.2688, Validation Loss: 0.5108
Step 580, Training Loss: 0.2498, Validation Loss: 0.5041
Epoch [15/15] finished
Finished Training


In [15]:
# Accuracy calculation for third set of parameters
load_checkpoint(save_path3, modified_model3, optimizer)
for threshold in range(0, 100, 5):
    accuracy = find_threshold(modified_model3, valid_loader, device, threshold/100)
    print('For threshold = {:.2f}, Accuracy = {:.4f}'.format(threshold/100, accuracy))

Model loaded from modify_net3.pt, with val loss: 0.47489261627197266
For threshold = 0.00, Accuracy = 64.0000
For threshold = 0.05, Accuracy = 82.7886
For threshold = 0.10, Accuracy = 85.2514
For threshold = 0.15, Accuracy = 87.4457
For threshold = 0.20, Accuracy = 88.2514
For threshold = 0.25, Accuracy = 90.4229
For threshold = 0.30, Accuracy = 90.7886
For threshold = 0.35, Accuracy = 91.4686
For threshold = 0.40, Accuracy = 92.2114
For threshold = 0.45, Accuracy = 91.7714
For threshold = 0.50, Accuracy = 90.6971
For threshold = 0.55, Accuracy = 87.8914
For threshold = 0.60, Accuracy = 87.2800
For threshold = 0.65, Accuracy = 85.3886
For threshold = 0.70, Accuracy = 83.4229
For threshold = 0.75, Accuracy = 78.3600
For threshold = 0.80, Accuracy = 72.5543
For threshold = 0.85, Accuracy = 66.8457
For threshold = 0.90, Accuracy = 59.9886
For threshold = 0.95, Accuracy = 58.5371


In [16]:
%reload_ext tensorboard
%tensorboard --logdir './runs'

Reusing TensorBoard on port 6006 (pid 18336), started 0:38:58 ago. (Use '!kill 18336' to kill it.)

In [17]:
# Q8C: Data Agumentation
# 1: Random Horizontal Flip with 0.5 probability
# 2: Random Rotation

new_transform1 = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),    
     transforms.Normalize((0.5,), (0.5,)) ])

new_transform2 = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.Compose([transforms.RandomRotation(degrees=(10, 90))]),
     transforms.ToTensor(),    
     transforms.Normalize((0.5,), (0.5,)) ])   

new_train_dataset = train_dataset + ImageDataset(PATH + 'train.csv', PATH + 'index.txt', new_transform1) + ImageDataset(PATH + 'train.csv', PATH + 'index.txt', new_transform2)
new_valid_dataset = valid_dataset + ImageDataset(PATH + 'valid.csv', PATH + 'index.txt', new_transform1) + ImageDataset(PATH + 'valid.csv', PATH + 'index.txt', new_transform2)

new_train_loader = DataLoader(new_train_dataset, batch_size=128, shuffle=True)
new_valid_loader = DataLoader(new_valid_dataset, batch_size=128, shuffle=False)

modified_model4 = Net().cuda()
# parameters obtained from Q8A third model
modified_num_epochs = 15
modified_optimizer = optim.Adam(modified_model4.parameters(), lr=0.00075)
modify_writer4 = SummaryWriter(os.path.join(PATH, 'runs/hr4'))
save_path4 = f'modify_net4.pt'

TRAIN(modified_model4, new_train_loader, new_valid_loader, modified_num_epochs, criterion, modified_optimizer, best_val_loss, device, save_path4, modify_writer4)

  0%|          | 0/115 [00:00<?, ?it/s]

Step 10, Training Loss: 0.6975, Validation Loss: 0.7007
Model saved to modify_net4.pt
Step 20, Training Loss: 0.6925, Validation Loss: 0.6848
Model saved to modify_net4.pt
Step 30, Training Loss: 0.6950, Validation Loss: 0.6910
Step 40, Training Loss: 0.6816, Validation Loss: 0.6909
Step 50, Training Loss: 0.6887, Validation Loss: 0.6895
Step 60, Training Loss: 0.6894, Validation Loss: 0.6875
Step 70, Training Loss: 0.6828, Validation Loss: 0.6935
Step 80, Training Loss: 0.6855, Validation Loss: 0.6901
Step 90, Training Loss: 0.6787, Validation Loss: 0.6850
Step 100, Training Loss: 0.6814, Validation Loss: 0.6816
Model saved to modify_net4.pt
Step 110, Training Loss: 0.6693, Validation Loss: 0.6863
Epoch [1/15] finished


  0%|          | 0/115 [00:00<?, ?it/s]

Step 120, Training Loss: 0.6969, Validation Loss: 0.6859
Step 130, Training Loss: 0.6895, Validation Loss: 0.6933
Step 140, Training Loss: 0.6759, Validation Loss: 0.6791
Model saved to modify_net4.pt
Step 150, Training Loss: 0.6701, Validation Loss: 0.6834
Step 160, Training Loss: 0.6756, Validation Loss: 0.6848
Step 170, Training Loss: 0.6747, Validation Loss: 0.6693
Model saved to modify_net4.pt
Step 180, Training Loss: 0.6756, Validation Loss: 0.6913
Step 190, Training Loss: 0.6780, Validation Loss: 0.6623
Model saved to modify_net4.pt
Step 200, Training Loss: 0.6596, Validation Loss: 0.6474
Model saved to modify_net4.pt
Step 210, Training Loss: 0.6533, Validation Loss: 0.6444
Model saved to modify_net4.pt
Step 220, Training Loss: 0.6487, Validation Loss: 0.6460
Step 230, Training Loss: 0.6297, Validation Loss: 0.6590
Epoch [2/15] finished


  0%|          | 0/115 [00:00<?, ?it/s]

...

Step 1390, Training Loss: 0.2822, Validation Loss: 0.4860
Step 1400, Training Loss: 0.2357, Validation Loss: 0.5175
Step 1410, Training Loss: 0.2800, Validation Loss: 0.5106
Step 1420, Training Loss: 0.2695, Validation Loss: 0.4759
Step 1430, Training Loss: 0.2645, Validation Loss: 0.5229
Step 1440, Training Loss: 0.2629, Validation Loss: 0.4703
Step 1450, Training Loss: 0.2554, Validation Loss: 0.5025
Step 1460, Training Loss: 0.2839, Validation Loss: 0.5428
Step 1470, Training Loss: 0.2739, Validation Loss: 0.4999
Step 1480, Training Loss: 0.2493, Validation Loss: 0.4860
Step 1490, Training Loss: 0.2473, Validation Loss: 0.4758
Epoch [13/15] finished


  0%|          | 0/115 [00:00<?, ?it/s]

Step 1500, Training Loss: 0.2371, Validation Loss: 0.5023
Step 1510, Training Loss: 0.2199, Validation Loss: 0.5252
Step 1520, Training Loss: 0.2175, Validation Loss: 0.5114
Step 1530, Training Loss: 0.2221, Validation Loss: 0.5252
Step 1540, Training Loss: 0.2322, Validation Loss: 0.5344
Step 1550, Training Loss: 0.2280, Validation Loss: 0.5191
Step 1560, Training Loss: 0.2200, Validation Loss: 0.5681
Step 1570, Training Loss: 0.2579, Validation Loss: 0.5059
Step 1580, Training Loss: 0.2496, Validation Loss: 0.5004
Step 1590, Training Loss: 0.2519, Validation Loss: 0.5363
Step 1600, Training Loss: 0.2843, Validation Loss: 0.4839
Step 1610, Training Loss: 0.2405, Validation Loss: 0.4932
Epoch [14/15] finished


  0%|          | 0/115 [00:00<?, ?it/s]

Step 1620, Training Loss: 0.2301, Validation Loss: 0.5217
Step 1630, Training Loss: 0.2189, Validation Loss: 0.5109
Step 1640, Training Loss: 0.2075, Validation Loss: 0.5224
Step 1650, Training Loss: 0.2304, Validation Loss: 0.5242
Step 1660, Training Loss: 0.2016, Validation Loss: 0.5147
Step 1670, Training Loss: 0.2009, Validation Loss: 0.5650
Step 1680, Training Loss: 0.2198, Validation Loss: 0.5568
Step 1690, Training Loss: 0.2349, Validation Loss: 0.5538
Step 1700, Training Loss: 0.2366, Validation Loss: 0.5144
Step 1710, Training Loss: 0.2045, Validation Loss: 0.5368
Step 1720, Training Loss: 0.2313, Validation Loss: 0.5382
Epoch [15/15] finished
Finished Training


In [18]:
load_checkpoint(save_path4, modified_model4, optimizer)
for threshold in range(0, 100, 5):
    accuracy = find_threshold(modified_model4, valid_loader, device, threshold/100)
    print('For threshold = {:.2f}, Accuracy = {:.4f}'.format(threshold/100, accuracy))

Model loaded from modify_net4.pt, with val loss: 0.4545670169241288
For threshold = 0.00, Accuracy = 64.0000
For threshold = 0.05, Accuracy = 84.5200
For threshold = 0.10, Accuracy = 89.1029
For threshold = 0.15, Accuracy = 92.9200
For threshold = 0.20, Accuracy = 93.2114
For threshold = 0.25, Accuracy = 93.6857
For threshold = 0.30, Accuracy = 94.5257
For threshold = 0.35, Accuracy = 94.6229
For threshold = 0.40, Accuracy = 94.0971
For threshold = 0.45, Accuracy = 93.5486
For threshold = 0.50, Accuracy = 93.5371
For threshold = 0.55, Accuracy = 91.0857
For threshold = 0.60, Accuracy = 89.6457
For threshold = 0.65, Accuracy = 87.8514
For threshold = 0.70, Accuracy = 86.5943
For threshold = 0.75, Accuracy = 83.7143
For threshold = 0.80, Accuracy = 80.2629
For threshold = 0.85, Accuracy = 75.2114
For threshold = 0.90, Accuracy = 71.1486
For threshold = 0.95, Accuracy = 60.6343


In [19]:
%reload_ext tensorboard
%tensorboard --logdir './runs'

Reusing TensorBoard on port 6006 (pid 18336), started 1:40:25 ago. (Use '!kill 18336' to kill it.)

### 4.2.6 Prediction
Finally, you have built your own face comparison model! The last step is to predict the similarity scores in test.csv and save the predictions as p1.csv. Your predictions should be aimed to produce an optimal accuracy score.

**[P1]**  Submit your predictions as p1.csv. The format of p1.csv should be the same as that of train.csv: first two columns for the image indices and the third column for the predicted label (1: match; 0: not match). It is important to keep the row order to be the same as that in test.csv.


In [ ]:
test_dataset = ImageDataset(PATH + 'test.csv', PATH + 'index.txt', transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

input1_list, input2_list, pred_list = [], [], []

optimal_threshold = 0.35

with torch.no_grad():
    modified_model4.eval()
    for input1, input2, _ in test_loader:
        outputs = modified_model4(input1,input2)
        preds = (outputs > 0.35)

        input1_list += (id1.tolist())
        input2_list += (id2.tolist())
        for pred in preds:
            if(pred == True):
                pred_list.append(1) 
            else:
                pred_list.append(0)

d = {
    'id1': input1_list,
    'id2': input2_list,
    'target': pred_list
} 
df = pd.DataFrame.from_dict(d)
df.to_csv('20599045_p1.csv',index=False)